# Passage Retreival

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize import sent_tokenize
import nltk

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
def group_sentences_into_passages(sentences, group_size=3):
    passages = [
        " ".join(sentences[i:i + group_size])
        for i in range(0, len(sentences), group_size)
    ]
    return passages

In [ ]:
import pandas as pd
xlsx_file = 'RECTIFY/Dataset_with_evidences.xlsx'
claims_df = pd.read_excel(xlsx_file)

In [ ]:
claims = claims_df['claim'].astype(str).tolist()

In [ ]:
# Load the passages from the text file
txt_file = 'RECTIFY/evidence_corpus.txt'  
with open(txt_file, 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
sentences = sent_tokenize(text)

In [ ]:
# Initialize the model
model_name = 'msmarco-distilbert-base-tas-b'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def get_highest_relevant_passage(query, sentences, group_size=4):
    # Group sentences into passages
    passages = group_sentences_into_passages(sentences, group_size)

    # Encode query and passages
    query_embedding = model.encode([query], convert_to_tensor=True)
    passage_embeddings = model.encode(passages, convert_to_tensor=True)

    # Compute cosine similarity between query and passages
    cosine_scores = util.cos_sim(query_embedding, passage_embeddings)

    # Find the index of the highest cosine similarity score
    highest_score_index = torch.argmax(cosine_scores)

    # Get the highest relevant passage
    highest_relevant_passage = passages[highest_score_index]
    return highest_relevant_passage

In [ ]:
df_new = pd.DataFrame()
lst1 = []
lst2 = []

In [ ]:
for query in claims:
    #print(query)
    highest_relevant_passage = get_highest_relevant_passage(query, sentences, group_size=4)
    lst1.append(query)
    lst2.append(highest_relevant_passage)

In [ ]:
df_new['claim'] = lst1
df_new['evidence'] = lst2

In [ ]:
output_file = 'RECTIFY/passage_results.xlsx'
df_new.to_excel(output_file, index=False)
print(f"Results saved to {output_file}")